# Import libraries

In [1]:
import warnings
import logging
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import random
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from PIL import Image

tf.get_logger().setLevel('INFO')
tf.autograph.set_verbosity(0)

tf.get_logger().setLevel(logging.ERROR)
tf.get_logger().setLevel('ERROR')
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)
device_name = tf.test.gpu_device_name()
if "GPU" not in device_name:
    print("GPU device not found")
print('Found GPU at: {}'.format(device_name))

2.6.4
Found GPU at: /device:GPU:0


In [3]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [4]:
print("--> Checking for physical Tensorflow devices")
for device in tf.config.list_physical_devices():
    print(": {}".format(device.name))

--> Checking for physical Tensorflow devices
: /physical_device:CPU:0
: /physical_device:GPU:0


# Load data

Solo questo dovrebbe essere diverso durante la challenge

In [5]:
dataset_dir = '/kaggle/input/homework1/training_data_final'
image_size = (96,96)

# Dataloader and Data Augmentation

In [6]:
dir(tf.keras.applications)

['DenseNet121',
 'DenseNet169',
 'DenseNet201',
 'EfficientNetB0',
 'EfficientNetB1',
 'EfficientNetB2',
 'EfficientNetB3',
 'EfficientNetB4',
 'EfficientNetB5',
 'EfficientNetB6',
 'EfficientNetB7',
 'InceptionResNetV2',
 'InceptionV3',
 'MobileNet',
 'MobileNetV2',
 'MobileNetV3Large',
 'MobileNetV3Small',
 'NASNetLarge',
 'NASNetMobile',
 'ResNet101',
 'ResNet101V2',
 'ResNet152',
 'ResNet152V2',
 'ResNet50',
 'ResNet50V2',
 'VGG16',
 'VGG19',
 'Xception',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_sys',
 'densenet',
 'efficientnet',
 'imagenet_utils',
 'inception_resnet_v2',
 'inception_v3',
 'mobilenet',
 'mobilenet_v2',
 'mobilenet_v3',
 'nasnet',
 'resnet',
 'resnet50',
 'resnet_v2',
 'vgg16',
 'vgg19',
 'xception']

In [7]:
from importlib import import_module

# Create the base model from the pre-trained model
IMG_SHAPE = image_size + (3,)
batch_size = 16
validation_split=0.1
model_name = 'EfficientNetB7'
layer_name = 'efficientnetb7'
cat_name = 'efficientnet'
base_model = getattr(tfk.applications, model_name)(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')
preprocess_input = getattr(tf.keras.applications, cat_name).preprocess_input


258088960/258076736 [==============================] - 9s 0us/step


In [8]:
noaug_noval_train_data_gen = tfk.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input)


aug_train_data_gen = tfk.preprocessing.image.ImageDataGenerator(
                                        rotation_range=180,
                                        height_shift_range=20,
                                        width_shift_range=20,
                                        zoom_range=0.5,
                                        shear_range = 40,
                                        horizontal_flip=True,
                                        vertical_flip=True, 
                                        fill_mode='reflect',
                                        validation_split=validation_split,
                                        zca_whitening = True,
                                        preprocessing_function=preprocess_input
                                        )

aug_validation_data_gen = tfk.preprocessing.image.ImageDataGenerator(
                                        preprocessing_function=preprocess_input,
                                        validation_split=validation_split)

train_gen = aug_train_data_gen.flow_from_directory(directory=dataset_dir,
                                               target_size=image_size,
                                               color_mode='rgb',
                                               class_mode='categorical',
                                               batch_size=batch_size,
                                               shuffle=True,
                                               seed=seed,
                                               subset='training')


validation_gen = aug_validation_data_gen.flow_from_directory(directory=dataset_dir,
                                               target_size=image_size,
                                               color_mode='rgb',
                                               class_mode='categorical',
                                               batch_size=batch_size,
                                               shuffle=True,
                                               seed=seed,
                                               subset='validation')

train_noval_gen_all = noaug_noval_train_data_gen.flow_from_directory(directory=dataset_dir,
                                               target_size=image_size,
                                               color_mode='rgb',
                                               class_mode='categorical',
                                               batch_size=batch_size,     
                                               shuffle=False,
                                               seed=seed)

Found 3191 images belonging to 8 classes.
Found 351 images belonging to 8 classes.
Found 3542 images belonging to 8 classes.


In [9]:
print("Assigned labels")
print(train_gen.class_indices)
print()
print("Target classes")
print(train_gen.classes)
num_classes = train_gen.num_classes

Assigned labels
{'Species1': 0, 'Species2': 1, 'Species3': 2, 'Species4': 3, 'Species5': 4, 'Species6': 5, 'Species7': 6, 'Species8': 7}

Target classes
[0 0 0 ... 7 7 7]


# Load pre-trained model for transfer learning

We can fine tune this with different pretrained models

In [10]:
base_model.summary()

Model: "efficientnetb7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 96, 96, 3)]  0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 96, 96, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 96, 96, 3)    7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 97, 97, 3)    0           normalization[0][0]              
_____________________________________________________________________________________

In [11]:
def build_model(input_shape, output_classes, learning_rate=0.001, freeze=True):
    input_layer = tfkl.Input(shape=input_shape, name='input_layer')

    # Freeze the base model
    if(freeze):
      base_model.trainable = False
    else:
      base_model.trainable = True
    # We need training=False for the BatchNormalization layer
    feature_extractor = base_model(input_layer , training=False)

    x = tfkl.GlobalAveragePooling2D()(feature_extractor)
    x = tfkl.Dropout(0.2, seed=seed)(x)
    x = tfkl.Dense(units=256, activation='swish', kernel_initializer=tfk.initializers.GlorotUniform(seed=seed), name='hidden_layer_1')(x)
    x = tfkl.Dropout(0.3, seed=seed)(x)
    output_layer = tfkl.Dense(units=output_classes, activation='softmax', kernel_initializer=tfk.initializers.GlorotUniform(seed=seed), name='output_layer')(x)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    # Compile the model
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(learning_rate=learning_rate), metrics='accuracy')

    # Return the model
    return model

In [12]:

def build_custom_model(input_shape, output_classes, learning_rate=0.001):
    chanDim = -1
    levels = 4
    input_layer = tfkl.Input(shape=input_shape, name='input_layer')
    x = tfkl.Conv2D(32, (7, 7), padding="same")(input_layer)
    x = tfkl.LeakyReLU(alpha=0.3)(x)
    x = tfkl.BatchNormalization(axis=chanDim)(x)
    x = tfkl.Add([input_layer,x])
    # CNN 
    for i in range(levels):
        first = tfkl.MaxPooling2D(pool_size=(3, 3))(x)
        x = tfkl.Conv2D(32*(i+2), (3, 3), padding="same")(first)
        x = tfkl.LeakyReLU(alpha=0.3)(x)
        x = tfkl.BatchNormalization(axis=chanDim)(x)
        x = tfkl.Add([first,x])
    x = tfkl.MaxPooling2D(pool_size=(3, 3))(x)
    
    # FC
    x = tfkl.GlobalAveragePooling2D()(x)
    x = tfkl.Dropout(0.3)(x)
    x = tfkl.Dense(units=256, activation='relu', kernel_initializer=tfk.initializers.GlorotUniform(seed=seed), name='hidden_layer_1')(x)
    x = tfkl.Dropout(0.3)(x)
    output_layer = tfkl.Dense(units=output_classes, activation='softmax', kernel_initializer=tfk.initializers.GlorotUniform(seed=seed), name='output_layer')(x)
    
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')
              
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(learning_rate=learning_rate),metrics="accuracy")
    return model

In [13]:
saved_as_dataset = False
if(saved_as_dataset):
    model = tfk.models.load_model(f'/kaggle/input/efficientnetb2/{model_name}_all.h5')   
else:
    model = build_model(IMG_SHAPE, num_classes, freeze=True)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     [(None, 96, 96, 3)]       0         
_________________________________________________________________
efficientnetb7 (Functional)  (None, 3, 3, 2560)        64097687  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2560)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2560)              0         
_________________________________________________________________
hidden_layer_1 (Dense)       (None, 256)               655616    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
output_layer (Dense)         (None, 8)                 2056  

In [14]:
  early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)

In [15]:
train_FC = True
if(train_FC):
    history = model.fit(
            x = train_gen,
            epochs = 50,
            validation_data = validation_gen,
            callbacks = [early_stopping_callback]
        ).history
    model.save(f"./models/{model_name}_FC_only.h5")

Epoch 1/50
200/200 [==============================] - 53s 155ms/step - loss: 1.7835 - accuracy: 0.3303 - val_loss: 1.4107 - val_accuracy: 0.4587
Epoch 2/50
200/200 [==============================] - 16s 79ms/step - loss: 1.5101 - accuracy: 0.4278 - val_loss: 1.3186 - val_accuracy: 0.4929
Epoch 3/50
200/200 [==============================] - 16s 82ms/step - loss: 1.4226 - accuracy: 0.4666 - val_loss: 1.3488 - val_accuracy: 0.4900
Epoch 4/50
200/200 [==============================] - 16s 78ms/step - loss: 1.3839 - accuracy: 0.4788 - val_loss: 1.3973 - val_accuracy: 0.4872
Epoch 5/50
200/200 [==============================] - 17s 83ms/step - loss: 1.3512 - accuracy: 0.5008 - val_loss: 1.2900 - val_accuracy: 0.4843
Epoch 6/50
200/200 [==============================] - 16s 80ms/step - loss: 1.3196 - accuracy: 0.5096 - val_loss: 1.2061 - val_accuracy: 0.5670
Epoch 7/50
200/200 [==============================] - 17s 82ms/step - loss: 1.3204 - accuracy: 0.4983 - val_loss: 1.1809 - val_accuracy

In [16]:
# Fine tune also the last 5/10 layers of the feature extraction part
train_CNN = True
model = tfk.models.load_model(f"./models/{model_name}_FC_only.h5")
if(train_CNN):
    leave_freeze = len(model.get_layer(layer_name).layers) - 10
    for layer in model.get_layer(layer_name).layers[:leave_freeze]:
       layer.trainable = False
    for layer in model.get_layer(layer_name).layers[leave_freeze:]:
       layer.trainable = True
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(),metrics="accuracy")
    model.summary()
    history = model.fit(
            x = train_gen,
            epochs = 30,
            validation_data = validation_gen,
            callbacks = [early_stopping_callback]
        ).history
    model.save(f"./models/{model_name}_CNN_tuned.h5")

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     [(None, 96, 96, 3)]       0         
_________________________________________________________________
efficientnetb7 (Functional)  (None, 3, 3, 2560)        64097687  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2560)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2560)              0         
_________________________________________________________________
hidden_layer_1 (Dense)       (None, 256)               655616    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
output_layer (Dense)         (None, 8)                 2056  

In [17]:
# Fine tune all the model
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
train_all = True
model = tfk.models.load_model(f"./models/{model_name}_CNN_tuned.h5")
if(train_all):
    for layer in model.get_layer(layer_name).layers:
       layer.trainable = True
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(learning_rate=2e-4),metrics="accuracy")
    model.summary()
    history = model.fit(
            x = train_gen,
            epochs = 200,
            validation_data = validation_gen,
            callbacks = [early_stopping_callback]
        ).history
    model.save(f"./models/{model_name}_all.h5")

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     [(None, 96, 96, 3)]       0         
_________________________________________________________________
efficientnetb7 (Functional)  (None, 3, 3, 2560)        64097687  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2560)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2560)              0         
_________________________________________________________________
hidden_layer_1 (Dense)       (None, 256)               655616    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
output_layer (Dense)         (None, 8)                 2056  

In [18]:
# Retrain also on validation with low learining rate
last_train = False
model = tfk.models.load_model(f"./models/{model_name}_all.h5")
if(last_train):
    for layer in model.get_layer(layer_name).layers:
       layer.trainable = True
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.SGD(learning_rate=1e-6),metrics="accuracy")
    model.summary()
    history = model.fit(
                x = validation_gen,
                epochs = 5
            ).history
    model.save(f"./models/{model_name}_final.h5")

# Create submission

In [19]:
with open("template.py", "w") as file:
  file.write(
f"""import os
import tensorflow as tf
class model:
    def __init__(self, path):
        self.model = tf.keras.models.load_model(os.path.join(path, f"SubmissionModel/model.h5"))
        self.preprocess_input = getattr(tf.keras.applications, '{cat_name}' ).preprocess_input
    def predict(self, X):
        X = self.preprocess_input(X)
        out = self.model.predict(X)
        out = tf.argmax(out, axis=-1)
        return out""")
!cat template.py

import os
import tensorflow as tf
class model:
    def __init__(self, path):
        self.model = tf.keras.models.load_model(os.path.join(path, f"SubmissionModel/model.h5"))
        self.preprocess_input = getattr(tf.keras.applications, 'efficientnet' ).preprocess_input
    def predict(self, X):
        X = self.preprocess_input(X)
        out = self.model.predict(X)
        out = tf.argmax(out, axis=-1)
        return out

In [20]:
! rm submission.zip
! rm -rf ./submission
! mkdir ./submission
! mkdir ./submission/SubmissionModel
! touch ./submission/metadata
! cp ./models/$model_name\_all.h5 ./submission/SubmissionModel/model.h5
! cp template.py ./submission/model.py
! zip -r submission.zip submission/*


rm: cannot remove 'submission.zip': No such file or directory
  adding: submission/SubmissionModel/ (stored 0%)
  adding: submission/SubmissionModel/model.h5 (deflated 6%)
  adding: submission/metadata (stored 0%)
  adding: submission/model.py (deflated 46%)


In [21]:
from IPython.display import FileLink
#FileLink(r'submission.zip')

# Test submission

In [22]:
from template import model
test_model = model('./submission')

In [23]:
image = Image.open('/kaggle/input/homework1/training_data_final/Species1/00006.jpg')
frame = np.expand_dims(np.asarray(image), 0)
pred = test_model.predict(train_noval_gen_all)

In [24]:
from sklearn.metrics import confusion_matrix, accuracy_score
print(accuracy_score(pred,train_noval_gen_all.labels ))
confusion_matrix(pred,train_noval_gen_all.labels )

0.9596273291925466


array([[170,   9,   6,   3,   1,   2,   0,  17],
       [  3, 502,   0,   0,   0,   0,   5,   8],
       [  0,   0, 483,   0,   2,   0,   0,   0],
       [  0,   2,   5, 502,   6,   0,   0,   1],
       [  0,   0,  19,   2, 520,   0,   0,   1],
       [  1,   4,   0,   0,   1, 217,   1,   2],
       [  3,   9,   0,   0,   0,   0, 531,   5],
       [  9,   6,   2,   4,   1,   3,   0, 474]])